<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/solver_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

Possibly need to restart runtime here.

In [1]:
# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

Cloning into 'Summer2021'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 40 (delta 8), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (40/40), done.


# 1. Load Data

In [1]:
#next run the model.py in the folder introScenario
!pwd
%cd ./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [4]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [5]:
inputs={'customerDemand':d_df,'plants':s_df}

In [14]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


{'output_lock': {}}
{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
0       1  ...                                       0
1       2  ...                                       0
2       3  ...                                       0
3       4  ...                                       0
4       5  ...                                       0
5       6  ...                                       0

[6 rows x 7 columns]}


In [16]:
exec(open("./model.py").read())

* building wado model
model building done in 0.06535887718200684 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model3 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all all

In [17]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       2  ...                                       0
 2       3  ...                                       0
 3       4  ...                                       0
 4       5  ...                                       0
 5       6  ...                                       0
 
 [6 rows x 7 columns]}

In [18]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,0.0,1.0,40,mask,3,0
1,2,15.0,1.0,30,mask,2,0
2,3,30.0,1.0,30,handSanitizer,1,0
3,4,70.0,1.0,100,handSanitizer,3,0
4,5,60.0,1.0,60,mask,2,0
5,6,45.0,1.0,45,mask,1,0


# 2. Solve the DO problem via DOcplex

In [ ]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model
# TODO

# 3. Solve the DO problem using IBM Quantum

In [ ]:
# TODO (after steps 1 and 2)